In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import cobra

from src.calc.ablation import get_ablation_ratio
from src.gem.yeast8model import Yeast8Model

# Construct models of cells of interest

In [ ]:
glc_exch_rate = 16.89
wt_ec = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
wt_ec.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt_ec.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

In [ ]:
from src.data.biomasscomponent import biomass_component_list_orig

glucose_bounds = (-4.75, 0) # gives a sensible growth rate for wt
wt_y8 = Yeast8Model("../data/gemfiles/yeast-GEM_8-6-0.xml", growth_id="r_2111", biomass_id="r_4041")
wt_y8.biomass_component_list = biomass_component_list_orig
wt_y8.model.reactions.r_1714.bounds = glucose_bounds
wt_y8.add_media_components(['r_1992'])

# Effect of flux constraints

Convenience functions

In [ ]:
def cf_vs_ablation(ymodel, fractions):
    # get original flux sum
    sol = ymodel.optimize()
    orig_flux_sum = sol.fluxes.abs().sum()
    
    abl_res_list = []
    for fraction in fractions:
        # impose flux constraint
        ub = fraction*orig_flux_sum
        ymodel.set_flux_constraint(upper_bound=ub)
        sol = ymodel.optimize()
        # ablate & ratio
        abl_res = ymodel.ablate()
        # append list
        abl_res_list.append(abl_res)
        
    return abl_res_list

## Parameters

In [ ]:
# Step down, because 'phantom' values or copying issues (that I don't want to fix)
fractions = np.linspace(1, 0, num=51)

## Enzyme-constrained

Get sum of absolute values in 'vanilla' FBA.

Then, sweep constraint value, see if it affects ablation.

Note: re-initialise the model before (re-)running these.

In [ ]:
# Generate results
abl_list = cf_vs_ablation(wt_ec, fractions)

In [ ]:
# Alternatively, load if saved
with open('../data/interim/ec_cf_vs_ablation.pkl', 'rb') as handle:
    abl_list = pickle.load(handle)

In [ ]:
# Dump data
with open('../data/interim/ec_cf_vs_ablation.pkl', 'wb') as handle:
    pickle.dump(abl_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Non-ec

In [ ]:
# Generate results
abl_list = cf_vs_ablation(wt_y8, fractions)

In [ ]:
# Alternatively, load if saved
with open('../data/interim/y8_cf_vs_ablation.pkl', 'rb') as handle:
    abl_list = pickle.load(handle)

In [ ]:
# Dump data
with open('../data/interim/y8_cf_vs_ablation.pkl', 'wb') as handle:
    pickle.dump(abl_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Plots

In [ ]:
# Define lists for plotting
gr_list = [abl_res.ablated_flux.iloc[0] for abl_res in abl_list]
ratio_list = [get_ablation_ratio(abl_res) for abl_res in abl_list]
abl_flux_list = [abl_res.ablated_flux.iloc[1:].to_list() for abl_res in abl_list]
abl_flux_array = np.array(abl_flux_list)
biomass_components_list = abl_list[0].priority_component.iloc[1:].to_list()

In [ ]:
fig, ax = plt.subplots()
ax.plot(fractions, gr_list, 'o-')
ax.set_xlabel('Fraction of original sum of absolute values of fluxes')
ax.set_ylabel('Growth rate')
ax.set_xlim((0,1))
ax.set_ylim((0,0.5))

In [ ]:
fig, ax = plt.subplots()
ax.plot(fractions, abl_flux_array, 'o-')
ax.set_xlabel('Fraction of original sum of absolute values of fluxes')
ax.set_ylabel('Ablated flux')
ax.set_xlim((0,1))
ax.set_ylim((0,1.5))
ax.legend(biomass_components_list)

In [ ]:
fig, ax = plt.subplots()
ax.plot(fractions, ratio_list, 'o-')
ax.set_xlabel('Fraction of original sum of absolute values of fluxes')
ax.set_ylabel('Ratio')
ax.set_xlim((0,1))
ax.set_ylim((0,1.4))